# 🏥 Healthcare Data Integration & ETL Pipeline
## HIPAA-Compliant Multi-Source Patient Data Warehouse

**Project Overview:**
Enterprise-grade ETL pipeline integrating patient data from Electronic Health Records (EHR), billing systems, and pharmacy data into a unified Snowflake data warehouse. Implements HIPAA compliance, data quality validation, and automated alerting.

**Business Problem:**
A regional healthcare network operates 15 hospitals with disparate systems:
- Legacy EHR systems (Epic, Cerner)
- Billing system (separate database)
- Pharmacy management system
- Patient portal data

They need:
1. Unified patient 360° view for care coordination
2. Real-time readmission risk analytics
3. HIPAA-compliant data handling
4. Automated data quality monitoring
5. Cost optimization and fraud detection

**Technical Solution:**
- **Extract:** AWS Glue jobs pulling from S3 (EHR exports), RDS (billing), API (pharmacy)
- **Transform:** PySpark data cleansing, deduplication, PHI masking
- **Load:** Incremental loads to Snowflake with SCD Type 2
- **Orchestration:** AWS Step Functions + EventBridge
- **Monitoring:** CloudWatch + SNS alerts
- **BI Layer:** Power BI dashboards with row-level security

---

**Author:** Pamela Austin | Senior Data Analyst (Data Engineering)  
**Date:** November 2025  
**Technologies:** AWS Glue, PySpark, Snowflake, Python, SQL, HIPAA Compliance

## 1. Project Architecture & Setup

### Data Sources:
1. **EHR System (Epic)** - Daily full exports to S3 (CSV/Parquet)
   - Patient demographics
   - Encounters/visits
   - Diagnoses (ICD-10)
   - Procedures (CPT codes)
   
2. **Billing System** - RDS PostgreSQL
   - Claims data
   - Insurance information
   - Payment history
   
3. **Pharmacy System** - REST API
   - Prescription orders
   - Medication dispensing
   - Drug interactions

### Target: Snowflake Data Warehouse
- **Database:** HEALTHCARE_DW
- **Schemas:** RAW, STAGING, CURATED, ANALYTICS
- **Security:** Role-based access control (RBAC), data masking

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import hashlib
import re
from faker import Faker
import warnings
warnings.filterwarnings('ignore')

# AWS SDK
import boto3
from botocore.exceptions import ClientError

# PySpark imports (for Glue)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Snowflake connector
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# Data quality
from great_expectations.dataset import PandasDataset

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Set styles
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Initialize Faker for generating synthetic healthcare data
fake = Faker()
Faker.seed(42)
np.random.seed(42)

print("✅ All libraries imported successfully!")
print(f"📅 Pipeline Run Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Generate Synthetic Healthcare Data

Creating realistic synthetic data that mirrors real healthcare scenarios while being HIPAA-compliant.

In [ ]:
# Generate synthetic patient data (EHR Source)
def generate_patient_data(n_patients=50000):
    """
    Generate synthetic patient demographics mimicking EHR system exports.
    Includes realistic distributions for age, gender, insurance, etc.
    """
    print(f"\n🏥 Generating {n_patients:,} synthetic patient records...")
    
    patients = []
    for i in range(n_patients):
        # Generate patient demographics
        gender = np.random.choice(['M', 'F', 'Other'], p=[0.48, 0.50, 0.02])
        birth_date = fake.date_of_birth(minimum_age=0, maximum_age=100)
        age = (datetime.now().date() - birth_date).days // 365
        
        # Insurance type distribution (realistic)
        if age >= 65:
            insurance_type = np.random.choice(['Medicare', 'Medicare Advantage', 'Private'], 
                                             p=[0.60, 0.25, 0.15])
        elif age < 26:
            insurance_type = np.random.choice(['Parent Plan', 'Medicaid', 'Private', 'Uninsured'],
                                             p=[0.40, 0.30, 0.20, 0.10])
        else:
            insurance_type = np.random.choice(['Private', 'Medicaid', 'Uninsured'],
                                             p=[0.60, 0.25, 0.15])
        
        patient = {
            'patient_id': f'PAT{str(i+1).zfill(8)}',
            'medical_record_number': f'MRN{str(i+1).zfill(10)}',
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'date_of_birth': birth_date,
            'age': age,
            'gender': gender,
            'ssn': fake.ssn(),  # PHI - will be encrypted
            'phone': fake.phone_number(),  # PHI
            'email': fake.email(),  # PHI
            'address': fake.street_address(),  # PHI
            'city': fake.city(),
            'state': fake.state_abbr(),
            'zip_code': fake.zipcode(),
            'insurance_type': insurance_type,
            'primary_care_physician': fake.name(),
            'registration_date': fake.date_between(start_date='-10y', end_date='today'),
            'last_visit_date': fake.date_between(start_date='-2y', end_date='today'),
            'is_active': np.random.choice([True, False], p=[0.85, 0.15]),
            'source_system': 'EPIC_EHR',
            'extract_timestamp': datetime.now()
        }
        patients.append(patient)
    
    df_patients = pd.DataFrame(patients)
    print(f"✅ Generated {len(df_patients):,} patient records")
    print(f"   • Active patients: {df_patients['is_active'].sum():,}")
    print(f"   • Average age: {df_patients['age'].mean():.1f} years")
    print(f"   • Medicare patients: {(df_patients['insurance_type'].str.contains('Medicare')).sum():,}")
    
    return df_patients

# Generate patient data
df_patients = generate_patient_data(50000)
df_patients.head(3)

In [ ]:
# Generate clinical encounters data
def generate_encounters(df_patients, avg_encounters_per_patient=3.5):
    """
    Generate hospital encounters/visits with realistic diagnosis and procedure codes.
    """
    print(f"\n🏥 Generating clinical encounters...")
    
    encounters = []
    encounter_types = ['Emergency', 'Outpatient', 'Inpatient', 'Observation', 'Telemedicine']
    encounter_type_probs = [0.15, 0.45, 0.20, 0.10, 0.10]
    
    # Common ICD-10 diagnosis codes
    common_diagnoses = [
        ('I10', 'Essential Hypertension'),
        ('E11.9', 'Type 2 Diabetes Mellitus'),
        ('J44.9', 'COPD'),
        ('I25.10', 'Coronary Artery Disease'),
        ('F41.9', 'Anxiety Disorder'),
        ('M79.3', 'Chronic Pain'),
        ('J06.9', 'Upper Respiratory Infection'),
        ('K21.9', 'GERD'),
        ('E78.5', 'Hyperlipidemia'),
        ('N18.3', 'Chronic Kidney Disease')
    ]
    
    for _, patient in df_patients.iterrows():
        # Number of encounters varies by age and insurance
        if patient['age'] > 65:
            n_encounters = int(np.random.poisson(avg_encounters_per_patient * 2))
        elif patient['age'] < 5:
            n_encounters = int(np.random.poisson(avg_encounters_per_patient * 1.5))
        else:
            n_encounters = int(np.random.poisson(avg_encounters_per_patient))
        
        for _ in range(max(1, n_encounters)):
            encounter_date = fake.date_between(
                start_date=patient['registration_date'],
                end_date='today'
            )
            
            encounter_type = np.random.choice(encounter_types, p=encounter_type_probs)
            
            # Length of stay varies by encounter type
            if encounter_type == 'Inpatient':
                los_days = int(np.random.lognormal(1.5, 0.8))  # Average ~5 days
            elif encounter_type == 'Observation':
                los_days = int(np.random.uniform(1, 3))
            else:
                los_days = 0
            
            discharge_date = encounter_date + timedelta(days=los_days) if los_days > 0 else encounter_date
            
            # Assign diagnoses (1-3 per encounter)
            n_diagnoses = np.random.choice([1, 2, 3], p=[0.50, 0.35, 0.15])
            diagnoses = np.random.choice(len(common_diagnoses), n_diagnoses, replace=False)
            primary_diagnosis = common_diagnoses[diagnoses[0]]
            
            encounter = {
                'encounter_id': f'ENC{fake.uuid4()[:10].upper()}',
                'patient_id': patient['patient_id'],
                'encounter_type': encounter_type,
                'admission_date': encounter_date,
                'discharge_date': discharge_date,
                'length_of_stay_days': los_days,
                'primary_diagnosis_code': primary_diagnosis[0],
                'primary_diagnosis_desc': primary_diagnosis[1],
                'attending_physician': fake.name(),
                'department': np.random.choice(['Cardiology', 'Endocrinology', 'Emergency', 
                                               'Primary Care', 'Pulmonology', 'Psychiatry']),
                'discharge_disposition': np.random.choice(['Home', 'Home with Services', 
                                                          'SNF', 'Rehab', 'AMA'], 
                                                         p=[0.70, 0.15, 0.08, 0.05, 0.02]),
                'source_system': 'EPIC_EHR',
                'extract_timestamp': datetime.now()
            }
            encounters.append(encounter)
    
    df_encounters = pd.DataFrame(encounters)
    print(f"✅ Generated {len(df_encounters):,} encounter records")
    print(f"   • Average encounters per patient: {len(df_encounters) / len(df_patients):.2f}")
    print(f"   • Inpatient encounters: {(df_encounters['encounter_type'] == 'Inpatient').sum():,}")
    print(f"   • Average LOS (inpatient): {df_encounters[df_encounters['length_of_stay_days'] > 0]['length_of_stay_days'].mean():.1f} days")
    
    return df_encounters

df_encounters = generate_encounters(df_patients, avg_encounters_per_patient=3.5)
df_encounters.head(3)

In [ ]:
# Generate billing/claims data
def generate_claims(df_encounters):
    """
    Generate billing claims data with charges, payments, and insurance processing.
    """
    print(f"\n💰 Generating billing claims data...")
    
    claims = []
    
    for _, encounter in df_encounters.iterrows():
        # Base charge varies by encounter type
        if encounter['encounter_type'] == 'Emergency':
            base_charge = np.random.uniform(1500, 8000)
        elif encounter['encounter_type'] == 'Inpatient':
            base_charge = np.random.uniform(10000, 75000)
        elif encounter['encounter_type'] == 'Outpatient':
            base_charge = np.random.uniform(200, 2500)
        elif encounter['encounter_type'] == 'Observation':
            base_charge = np.random.uniform(3000, 12000)
        else:  # Telemedicine
            base_charge = np.random.uniform(50, 250)
        
        # Additional charges for LOS
        total_charge = base_charge + (encounter['length_of_stay_days'] * np.random.uniform(2000, 5000))
        
        # Insurance payment (typically 70-90% of charges)
        insurance_payment = total_charge * np.random.uniform(0.70, 0.90)
        
        # Patient responsibility
        patient_responsibility = total_charge - insurance_payment
        patient_payment = patient_responsibility * np.random.uniform(0.60, 0.95)  # Some unpaid
        
        # Claim status
        claim_status = np.random.choice(['Paid', 'Pending', 'Denied', 'Appealed'], 
                                       p=[0.85, 0.08, 0.05, 0.02])
        
        claim = {
            'claim_id': f'CLM{fake.uuid4()[:10].upper()}',
            'encounter_id': encounter['encounter_id'],
            'patient_id': encounter['patient_id'],
            'claim_date': encounter['discharge_date'],
            'total_charges': round(total_charge, 2),
            'insurance_payment': round(insurance_payment, 2),
            'patient_responsibility': round(patient_responsibility, 2),
            'patient_payment': round(patient_payment, 2),
            'outstanding_balance': round(patient_responsibility - patient_payment, 2),
            'claim_status': claim_status,
            'billing_provider': fake.company(),
            'payment_date': encounter['discharge_date'] + timedelta(days=int(np.random.uniform(30, 120))),
            'source_system': 'BILLING_DB',
            'extract_timestamp': datetime.now()
        }
        claims.append(claim)
    
    df_claims = pd.DataFrame(claims)
    print(f"✅ Generated {len(df_claims):,} billing claims")
    print(f"   • Total charges: ${df_claims['total_charges'].sum():,.2f}")
    print(f"   • Total insurance payments: ${df_claims['insurance_payment'].sum():,.2f}")
    print(f"   • Outstanding balance: ${df_claims['outstanding_balance'].sum():,.2f}")
    print(f"   • Claim denial rate: {(df_claims['claim_status'] == 'Denied').mean() * 100:.1f}%")
    
    return df_claims

df_claims = generate_claims(df_encounters)
df_claims.head(3)

In [ ]:
# Generate pharmacy/medication data
def generate_prescriptions(df_patients, avg_meds_per_patient=2.5):
    """
    Generate prescription and medication dispensing data.
    """
    print(f"\n💊 Generating prescription data...")
    
    # Common medications
    medications = [
        ('Lisinopril', 'Antihypertensive', 10.50),
        ('Metformin', 'Antidiabetic', 8.25),
        ('Atorvastatin', 'Statin', 15.75),
        ('Omeprazole', 'PPI', 12.00),
        ('Albuterol', 'Bronchodilator', 25.50),
        ('Levothyroxine', 'Thyroid', 9.00),
        ('Amlodipine', 'Calcium Channel Blocker', 11.25),
        ('Metoprolol', 'Beta Blocker', 10.00),
        ('Gabapentin', 'Anticonvulsant', 18.75),
        ('Sertraline', 'SSRI', 22.50)
    ]
    
    prescriptions = []
    
    for _, patient in df_patients.iterrows():
        # Patients over 50 tend to have more prescriptions
        if patient['age'] > 65:
            n_meds = int(np.random.poisson(avg_meds_per_patient * 2))
        elif patient['age'] > 50:
            n_meds = int(np.random.poisson(avg_meds_per_patient * 1.5))
        else:
            n_meds = int(np.random.poisson(avg_meds_per_patient * 0.8))
        
        for _ in range(max(0, n_meds)):
            medication = medications[np.random.randint(0, len(medications))]
            
            prescription_date = fake.date_between(
                start_date=patient['registration_date'],
                end_date='today'
            )
            
            # Days supply and refills
            days_supply = np.random.choice([30, 60, 90], p=[0.60, 0.25, 0.15])
            refills_allowed = np.random.choice([0, 1, 2, 3, 11], p=[0.10, 0.20, 0.30, 0.25, 0.15])
            refills_remaining = np.random.randint(0, refills_allowed + 1)
            
            prescription = {
                'prescription_id': f'RX{fake.uuid4()[:10].upper()}',
                'patient_id': patient['patient_id'],
                'medication_name': medication[0],
                'medication_class': medication[1],
                'prescribing_physician': fake.name(),
                'prescription_date': prescription_date,
                'fill_date': prescription_date + timedelta(days=np.random.randint(0, 7)),
                'days_supply': days_supply,
                'quantity': np.random.randint(30, 180),
                'refills_allowed': refills_allowed,
                'refills_remaining': refills_remaining,
                'pharmacy_name': np.random.choice(['CVS', 'Walgreens', 'Walmart', 'Kroger', 'Local Pharmacy']),
                'cost': round(medication[2] * (days_supply / 30), 2),
                'copay': round(np.random.uniform(5, 50), 2),
                'is_generic': np.random.choice([True, False], p=[0.85, 0.15]),
                'source_system': 'PHARMACY_API',
                'extract_timestamp': datetime.now()
            }
            prescriptions.append(prescription)
    
    df_prescriptions = pd.DataFrame(prescriptions)
    print(f"✅ Generated {len(df_prescriptions):,} prescription records")
    print(f"   • Average prescriptions per patient: {len(df_prescriptions) / len(df_patients):.2f}")
    print(f"   • Generic medication rate: {df_prescriptions['is_generic'].mean() * 100:.1f}%")
    print(f"   • Total medication cost: ${df_prescriptions['cost'].sum():,.2f}")
    
    return df_prescriptions

df_prescriptions = generate_prescriptions(df_patients, avg_meds_per_patient=2.5)
df_prescriptions.head(3)

## 3. Data Quality Assessment (Pre-ETL)

Before loading data, we perform comprehensive quality checks to identify:
- Missing critical fields
- Data type mismatches
- Duplicate records
- Referential integrity issues
- Outliers and anomalies

In [ ]:
# Data Quality Framework
def assess_data_quality(df, table_name, critical_columns):
    """
    Comprehensive data quality assessment with detailed reporting.
    """
    print(f"\n🔍 Data Quality Assessment: {table_name}")
    print("=" * 80)
    
    quality_report = {
        'table_name': table_name,
        'total_records': len(df),
        'total_columns': len(df.columns),
        'assessment_date': datetime.now(),
        'issues': []
    }
    
    # 1. Completeness Check
    print("\n📊 Completeness Analysis:")
    missing_summary = df.isnull().sum()
    missing_pct = (missing_summary / len(df) * 100).round(2)
    
    for col in critical_columns:
        if col in df.columns:
            missing_count = missing_summary[col]
            if missing_count > 0:
                issue = f"Critical column '{col}' has {missing_count} ({missing_pct[col]}%) missing values"
                quality_report['issues'].append(issue)
                print(f"   ⚠️  {issue}")
            else:
                print(f"   ✅ {col}: 100% complete")
    
    # 2. Duplicate Check
    print("\n🔄 Duplicate Analysis:")
    if 'patient_id' in df.columns:
        duplicates = df['patient_id'].duplicated().sum()
        if duplicates > 0:
            issue = f"Found {duplicates} duplicate patient_id records"
            quality_report['issues'].append(issue)
            print(f"   ⚠️  {issue}")
        else:
            print(f"   ✅ No duplicate patient_id found")
    
    # 3. Data Type Validation
    print("\n📝 Data Type Validation:")
    print(df.dtypes.value_counts())
    
    # 4. Statistical Summary
    print("\n📈 Statistical Summary:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(df[numeric_cols].describe().round(2))
    
    # 5. Overall Quality Score
    completeness_score = (1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    duplicate_score = (1 - (duplicates / len(df) if 'patient_id' in df.columns else 0)) * 100
    overall_score = (completeness_score + duplicate_score) / 2
    
    quality_report['completeness_score'] = round(completeness_score, 2)
    quality_report['overall_quality_score'] = round(overall_score, 2)
    
    print(f"\n🎯 Quality Score: {overall_score:.1f}%")
    print(f"   • Completeness: {completeness_score:.1f}%")
    print(f"   • Uniqueness: {duplicate_score:.1f}%")
    
    if len(quality_report['issues']) == 0:
        print("\n✅ No critical issues found!")
    else:
        print(f"\n⚠️  {len(quality_report['issues'])} issue(s) identified")
    
    return quality_report

# Assess quality for each dataset
quality_patients = assess_data_quality(
    df_patients, 
    'PATIENTS', 
    ['patient_id', 'medical_record_number', 'date_of_birth', 'gender']
)

quality_encounters = assess_data_quality(
    df_encounters,
    'ENCOUNTERS',
    ['encounter_id', 'patient_id', 'admission_date', 'encounter_type']
)

quality_claims = assess_data_quality(
    df_claims,
    'CLAIMS',
    ['claim_id', 'encounter_id', 'patient_id', 'total_charges']
)

quality_prescriptions = assess_data_quality(
    df_prescriptions,
    'PRESCRIPTIONS',
    ['prescription_id', 'patient_id', 'medication_name', 'prescription_date']
)

## 4. HIPAA Compliance & PHI Encryption

Implementing HIPAA-compliant data handling:
- **PHI Encryption:** Hash SSN, mask phone/email
- **De-identification:** Create anonymous identifiers
- **Audit Logging:** Track all data access
- **Access Control:** Role-based security

In [ ]:
# HIPAA Compliance Functions
def hash_phi(value):
    """
    One-way hash for PHI fields (SSN, etc.) using SHA-256.
    In production, use AWS KMS or similar encryption service.
    """
    if pd.isna(value):
        return None
    return hashlib.sha256(str(value).encode()).hexdigest()

def mask_phone(phone):
    """
    Mask phone number: (555) 123-4567 -> (XXX) XXX-4567
    """
    if pd.isna(phone):
        return None
    # Keep last 4 digits only
    cleaned = re.sub(r'\D', '', phone)
    if len(cleaned) >= 4:
        return 'XXX-XXX-' + cleaned[-4:]
    return 'XXX-XXX-XXXX'

def mask_email(email):
    """
    Mask email: john.doe@example.com -> j***e@example.com
    """
    if pd.isna(email):
        return None
    parts = email.split('@')
    if len(parts) == 2:
        username = parts[0]
        if len(username) > 2:
            return f"{username[0]}{'*' * (len(username)-2)}{username[-1]}@{parts[1]}"
    return 'masked@domain.com'

def create_deidentified_id(patient_id, salt='HEALTHCARE_DW_2025'):
    """
    Create consistent de-identified patient ID for analytics.
    """
    combined = f"{patient_id}_{salt}"
    hashed = hashlib.sha256(combined.encode()).hexdigest()[:16]
    return f"DEID_{hashed.upper()}"

# Apply HIPAA transformations
def apply_hipaa_compliance(df_patients):
    """
    Apply HIPAA-compliant transformations to patient data.
    """
    print("\n🔒 Applying HIPAA Compliance Transformations...")
    
    df_secure = df_patients.copy()
    
    # Create de-identified patient ID
    df_secure['deidentified_patient_id'] = df_secure['patient_id'].apply(create_deidentified_id)
    
    # Hash SSN (one-way encryption)
    df_secure['ssn_hash'] = df_secure['ssn'].apply(hash_phi)
    df_secure = df_secure.drop('ssn', axis=1)
    
    # Mask phone and email
    df_secure['phone_masked'] = df_secure['phone'].apply(mask_phone)
    df_secure['email_masked'] = df_secure['email'].apply(mask_email)
    df_secure = df_secure.drop(['phone', 'email'], axis=1)
    
    # Generalize address to ZIP code only for analytics
    df_secure['address_city_state'] = df_secure['city'] + ', ' + df_secure['state']
    df_secure = df_secure.drop(['address', 'city', 'state'], axis=1)
    
    # Add compliance metadata
    df_secure['phi_encrypted_date'] = datetime.now()
    df_secure['compliance_version'] = 'HIPAA_2024_v1'
    
    print("✅ HIPAA compliance applied:")
    print(f"   • SSN: Hashed (SHA-256)")
    print(f"   • Phone: Masked (last 4 digits only)")
    print(f"   • Email: Masked")
    print(f"   • Address: Generalized to city/state")
    print(f"   • De-identified ID created for {len(df_secure):,} patients")
    
    return df_secure

# Apply compliance transformations
df_patients_secure = apply_hipaa_compliance(df_patients)

# Display comparison
print("\n📋 Before HIPAA Compliance:")
print(df_patients[['patient_id', 'first_name', 'last_name', 'ssn', 'phone', 'email', 'address']].head(2))

print("\n🔒 After HIPAA Compliance:")
print(df_patients_secure[['patient_id', 'deidentified_patient_id', 'first_name', 'last_name', 
                          'ssn_hash', 'phone_masked', 'email_masked', 'address_city_state']].head(2))